<a href="https://colab.research.google.com/github/newfull5/AI_Poet-KoGPT2/blob/master/2.%20Traing!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!git clone https://github.com/SKT-AI/KoGPT2.git

In [0]:
!cd KoGPT2 && pip install -r requirements.txt
!cd KoGPT2 && pip install .

In [0]:
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from torch.utils.data import DataLoader
import torch

In [5]:
if torch.cuda.device_count():
  print('구글 만세!')
  PU = 'cuda'
else:
  PU = 'cpu'

구글 만세!


In [0]:
#토큰화와 인덱싱을해서 리턴하는 함수

def dataset (file_path):
  data = []
  tokenizer = SentencepieceTokenizer(get_tokenizer())
  f = open(file_path,'r',encoding='utf-8')

  while True:
    file = f.readline()

    if not file:
      break
    line = tokenizer(file)
    indexing_word = [vocab[vocab.bos_token]]+ vocab[line] + [vocab[vocab.eos_token]]
    data.append(indexing_word)

  f.close()

  return data

In [0]:
model, vocab = get_pytorch_kogpt2_model()

model.to(torch.device(PU)) #모델 연산 유닛 설정
model.train() #모델 학습모드로 변경


In [0]:
save_path = 'drive/My Drive/Colab Notebooks/KoGPT2_checkpoint/'

#torch.save(model.state_dict,'model_state_dict.pth') #모델의 가중치 값을 저장하는 코드입니다.
#model.load_state_dict(torch.load('model_state_dict.pth')) #모델의 가중치 값을 불러오는 코드입니다.

#torch.save(model, PATH) #모델 전체를 저장하는 코드입니다.
#model = torch.load(PATH) #모델 전체를 불러오는 코드입니다.

In [9]:
file_path = 'drive/My Drive/poem_data/korean_poem1~8.txt'
dataset = DataLoader(dataset(file_path), batch_size=100, shuffle=True, pin_memory=True)

learning_rate = 0.00005
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

using cached model


In [11]:
for epoch in range(epochs):
  cnt = 0

  for data in dataset:
    optimizer.zero_grad()

    data = torch.stack(data)
    data = data.transpose(1,0)
    data = data.to(PU)

    output = model(data,labels=data)
    loss, logits = output[:2]
    loss.backward()
    optimizer.step()

    cnt += 1
    
  if cnt % 50 == 0:
    print("[+] epoch : {}, cnt : {}, loss : {}".format(epoch, cnt+1, loss))

  if cnt % 200 == 0:
    torch.save({
        'epoch': epoch,
        'cnt': cnt,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, save_path+'KoGPT2_checkpoint')
  

[+] epoch : 0, cnt : 1, loss : 6.619760513305664
[+] epoch : 1, cnt : 1, loss : 5.794950485229492


KeyboardInterrupt: ignored

In [0]:
model